In [1]:
import pandas as pd
import numpy as np

In [2]:
csv_data = pd.read_csv(r'CleanData.csv');

In [3]:
trials = range(1,41)
bpm = [80,100,120]
gestureSize = ["S","M","L"]

In [5]:
csv_data.DistanceCoveredSoFar = csv_data.DistanceCoveredSoFar.astype(float)
csv_data.VelocityMagnitude = csv_data.VelocityMagnitude.astype(float)
csv_data.AngleToBP1 = csv_data.AngleToBP1.astype(float)
csv_data.BPM = csv_data.BPM.astype(int)
csv_data.Position_Y = csv_data.Position_Y.astype(float)

In [6]:
newOutputTable = pd.DataFrame({'Trial':[],'BPM':[],'GestureSize':[], 'MinAngleRegionTwo':[]})

In [8]:
# Here's where we iterate over the arrays we mentioned earlier
for g in gestureSize:
    for b in bpm:
        for i in trials: 
            sliceByTrial = csv_data.loc[csv_data.Trial == i] 
            sliceByGesture = sliceByTrial.loc[sliceByTrial.GestureSize == g] 
            sliceByBPM = sliceByGesture.loc[sliceByGesture.BPM == b] 
            if(sliceByBPM.Trial.size == 0):
               continue
            
            # =======================================
            # ============ REGION 1 =================
            # =======================================
            
            globalHighestPointIndex = sliceByBPM.Position_Y.idxmax()
            
            localHighestPointIndex = globalHighestPointIndex - sliceByBPM.index[0] + 1
            
            sliceToGetPositiveYVelocity = sliceByBPM.loc[sliceByBPM.Velocity_Y > 0]
            
            globalIndexFirstPositiveYVelocity = sliceToGetPositiveYVelocity.index[0]
            
            localIndexFirstPositiveYVelocity = globalIndexFirstPositiveYVelocity - sliceByBPM.index[0]
            
            RegionOne = sliceByBPM[localIndexFirstPositiveYVelocity:localHighestPointIndex+1] 
            
            # =======================================
            # ============ REGION 2 =================
            # =======================================
            
            smallestYPoint_RegionOneIndex = RegionOne.Position_Y.idxmin()
            
            y_positionHighestPoint = sliceByBPM.Position_Y[globalHighestPointIndex]
            
            y_positionLowestPoint  = sliceByBPM.Position_Y[smallestYPoint_RegionOneIndex]
            
            y_distance = np.absolute(y_positionHighestPoint - y_positionLowestPoint)
            
            y_distanceWithinRegion2 = y_distance/2
            
            portionAfterRegionOne = sliceByBPM[localHighestPointIndex+1:]
            
            RegionTwo = portionAfterRegionOne[np.absolute(portionAfterRegionOne.Position_Y - y_positionHighestPoint) < y_distanceWithinRegion2]
            
            # =======================================
            # ========== MIN ANGLE R2 ===============
            # =======================================
                    
            minAngleRegionTwo = min(RegionTwo["AngleToBP1"])
            newOutputTable = newOutputTable.append({'Trial': i,'BPM':b,'GestureSize':g,'MinAngleRegionTwo':minAngleRegionTwo}, ignore_index = True)

In [9]:
newOutputTable

,Trial,BPM,GestureSize,MinAngleRegionTwo
0,1.0,80.0,S,67.03144
1,2.0,80.0,S,66.94464
2,3.0,80.0,S,61.58102
3,4.0,80.0,S,75.72021
4,5.0,80.0,S,65.42288
5,6.0,80.0,S,63.35981
6,7.0,80.0,S,69.66460
7,8.0,80.0,S,64.22867
8,9.0,80.0,S,65.47829
9,10.0,80.0,S,66.58222


In [11]:
newOutputTable.to_csv(r'MinAngleRegionTwo.csv')